# MUC16 Mutation Trans Effect on Proteomics

This notebook analyzes the trans effect of MUC16 mutation on interacting and other proteins, in Endometrial, Colon, and Ovarian cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()
ov = cptac.Ovarian()

### Specify Gene

In [2]:
gene = "MUC16"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
omics = "proteomics"
#omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Proteomics

### Generate interacting protein list

Make a call to the cptac.algorithms get interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene

In [6]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
MUC17
ST6GAL1
ST6GALNAC2
ST3GAL3
C1GALT1C1
ST6GALNAC3
ST6GALNAC4
MUC21
MUC3A
ST3GAL2
MUC16
MSLN
C1GALT1
MUC6
MUC7
MUC20
MUC15
MUC4
ST3GAL1
ST3GAL4
GALNT4
MUC5B
MUC12
MUC1
MUC13
MUC5AC
LGALS1
IGBP1
UBC
RAD21
CEP170P1
MCM2
NANOG
POU5F1
EGFR
TRIM25
KIAA1429


## Endometrial

### Test for significant comparisons in any of interacting proteins

In [7]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Endometrial", True, all_significant_comparisons)


MUC17 did not match any columns in proteomics dataframe. MUC17_proteomics column inserted, but filled with NaN.
ST3GAL3 did not match any columns in proteomics dataframe. ST3GAL3_proteomics column inserted, but filled with NaN.
ST6GALNAC4 did not match any columns in proteomics dataframe. ST6GALNAC4_proteomics column inserted, but filled with NaN.
MUC21 did not match any columns in proteomics dataframe. MUC21_proteomics column inserted, but filled with NaN.
MUC3A did not match any columns in proteomics dataframe. MUC3A_proteomics column inserted, but filled with NaN.
MUC7 did not match any columns in proteomics dataframe. MUC7_proteomics column inserted, but filled with NaN.
MUC20 did not match any columns in proteomics dataframe. MUC20_proteomics column inserted, but filled with NaN.
MUC15 did not match any columns in proteomics dataframe. MUC15_proteomics column inserted, but filled with NaN.
MUC4 did not match any columns in proteomics dataframe. MUC4_proteomics column inserted, but

## Colon

### Test for significant comparisons in any of interacting proteins

In [8]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Colon", True, all_significant_comparisons)


MUC17 did not match any columns in proteomics dataframe. MUC17_proteomics column inserted, but filled with NaN.
ST6GALNAC2 did not match any columns in proteomics dataframe. ST6GALNAC2_proteomics column inserted, but filled with NaN.
ST3GAL3 did not match any columns in proteomics dataframe. ST3GAL3_proteomics column inserted, but filled with NaN.
ST6GALNAC3 did not match any columns in proteomics dataframe. ST6GALNAC3_proteomics column inserted, but filled with NaN.
ST6GALNAC4 did not match any columns in proteomics dataframe. ST6GALNAC4_proteomics column inserted, but filled with NaN.
MUC21 did not match any columns in proteomics dataframe. MUC21_proteomics column inserted, but filled with NaN.
MUC3A did not match any columns in proteomics dataframe. MUC3A_proteomics column inserted, but filled with NaN.
ST3GAL2 did not match any columns in proteomics dataframe. ST3GAL2_proteomics column inserted, but filled with NaN.
MUC16 did not match any columns in proteomics dataframe. MUC16_pro

## Ovarian

### Test for significant comparisons in any of interacting proteins

In [9]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']


'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.loc[ind,'Label'] = 'Mutated'
    else:
        protdf.loc[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)
#protdf = protdf.drop_duplicates()
protdf = protdf.loc[:,~protdf.columns.duplicated()]


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")


'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Ovarian", True, all_significant_comparisons)



MUC17 did not match any columns in proteomics dataframe. MUC17_proteomics column inserted, but filled with NaN.
ST6GALNAC2 did not match any columns in proteomics dataframe. ST6GALNAC2_proteomics column inserted, but filled with NaN.
ST3GAL3 did not match any columns in proteomics dataframe. ST3GAL3_proteomics column inserted, but filled with NaN.
ST6GALNAC3 did not match any columns in proteomics dataframe. ST6GALNAC3_proteomics column inserted, but filled with NaN.
ST6GALNAC4 did not match any columns in proteomics dataframe. ST6GALNAC4_proteomics column inserted, but filled with NaN.
MUC21 did not match any columns in proteomics dataframe. MUC21_proteomics column inserted, but filled with NaN.
MUC3A did not match any columns in proteomics dataframe. MUC3A_proteomics column inserted, but filled with NaN.
ST3GAL2 did not match any columns in proteomics dataframe. ST3GAL2_proteomics column inserted, but filled with NaN.
MUC6 did not match any columns in proteomics dataframe. MUC6_prote

# All Proteins: Proteomics

## Endometrial

In [10]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Endometrial", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  MUC16
Doing t-test comparisons

          Comparison   P_Value
0  ZNF362_proteomics  0.000003





## Colon

In [11]:
try:
    print("\nGene: ", gene)
    
    '''Use all proteins'''
    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Colon", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  MUC16
Doing t-test comparisons

           Comparison   P_Value
0   HSPA4L_proteomics  0.000002
1  CEACAM5_proteomics  0.000003
2   SMCHD1_proteomics  0.000004
3   TRMT2A_proteomics  0.000006





## Ovarian

In [12]:
print("\nGene: ", gene)

'''Use all proteins'''
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']


'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.loc[ind,'Label'] = 'Mutated'
    else:
        protdf.loc[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)
protdf = protdf.loc[:,~protdf.columns.duplicated()]


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Ovarian", False, all_significant_comparisons)



Gene:  MUC16
Doing t-test comparisons

No significant comparisons.


### Print all significant comparisons

In [13]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Colon,MUC16,MCM2_proteomics,True,0.000621
0,Endometrial,MUC16,ZNF362_proteomics,False,0.000003
0,Colon,MUC16,HSPA4L_proteomics,False,0.000002
1,Colon,MUC16,CEACAM5_proteomics,False,0.000003
2,Colon,MUC16,SMCHD1_proteomics,False,0.000004
3,Colon,MUC16,TRMT2A_proteomics,False,0.000006


### Write significant comparisons (if any) to shared CSV file

In [14]:
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)